## Chat Model Messages: An Overview

This document details how messages are structured and used within chat models, particularly focusing on how LangChain handles them.

### Core Concepts

*   **Messages** are the basic units of communication in chat models, representing both inputs and outputs.
*   Each message has a **`role`** and **`content`**, along with optional metadata.
*   **LangChain** provides a consistent way to manage messages across different chat models.

### Message Components

*   **`Role`**:  This categorises the message and helps the chat model respond appropriately. Key roles include:
    *   **`system`**: Used to instruct the model and provide context.
    *   **`user`**: Represents input from a user.
    *   **`assistant`**: Represents a response from the model.
    *   **`tool`**: Passes results of a tool invocation back to the model.
    *   `function`: A legacy role for OpenAI's function-calling API, `tool` should be used instead.
*   **`Content`**: The actual message information, can be text or multimodal data (images, audio, video). Most models primarily use text.
*   **Other Data**: Messages may include:
    *   **`ID`**: A unique identifier.
    *   **`Name`**: Differentiates entities with the same role.
    *   **`Metadata`**: Additional info, like timestamps or token usage.
    *   **`Tool Calls`**: Requests from the model to use tools.

### Conversation Structure

*   A typical conversation consists of a structured sequence of messages, usually alternating between **`user`** and **`assistant`** messages.

### LangChain Message Types

*   LangChain represents messages as Python objects inheriting from `BaseMessage`.
*   Key message types:
    *   **`SystemMessage`**: Corresponds to the `system` role.
    *   **`HumanMessage`**: Corresponds to the `user` role.
    *   **`AIMessage`**: Corresponds to the `assistant` role.
    *   **`AIMessageChunk`**: Used for streaming responses, also `assistant` role.
    *   **`ToolMessage`**: Corresponds to the `tool` role.
*   Other message types:
    *   **`RemoveMessage`**: Manages chat history, not associated with any role.
    *   **`FunctionMessage`**: Legacy message for the `function` role, use `ToolMessage` instead.

### Specific Message Details

*   **`SystemMessage`**:
    *   Primes the AI model by providing context.
    *   Implemented differently by providers (message role, API parameter, or no support).
    *   LangChain adapts based on provider capabilities and tries to incorporate the content in a HumanMessage if system messages are not supported by the provider.
*   **`HumanMessage`**:
    *   Represents user input, usually text, sometimes multimodal content.
    *   LangChain can convert a string input into a `HumanMessage`.
*   **`AIMessage`**:
    *   Model's response, may include text or tool calls.
     *  The `content` property is not standardized and can be text or a list of dictionaries.
    *   Includes attributes like `content`, `tool_calls`, `invalid_tool_calls`, `usage_metadata`, `id`, and `response_metadata`.
*   **`AIMessageChunk`**:
    *   For streaming responses.
    *   Can be merged into a single `AIMessage` using the `+` operator.
*   **`ToolMessage`**:
    *   Contains the result of tool calls, with a `tool_call_id` and an `artifact` field.
*   **`RemoveMessage`**: For managing chat history in LangGraph.
*   **`(Legacy) FunctionMessage`**: For the legacy function-calling API. `ToolMessage` should be used instead.

### OpenAI Format

*   Chat models accept OpenAI's format as input, a dictionary with `role` and `content`.
*   The output of models will be in terms of LangChain messages which can be converted to OpenAI format using `convert_to_openai_messages`.


In [ ]:
from langchain_core.messages import HumanMessage
from langchain_ollama.chat_models import ChatOllama

model = ChatOllama(model='llama3.1')

model.invoke([HumanMessage(content="Hello, how are you?")])

AIMessage(content="I'm just a computer program, so I don't have feelings like humans do, but thank you for asking! I'm functioning properly and ready to help with any questions or tasks you may have. How about you? How's your day going so far?", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-12-17T19:03:11.493834854Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1758532857, 'load_duration': 1316866053, 'prompt_eval_count': 16, 'prompt_eval_duration': 55000000, 'eval_count': 53, 'eval_duration': 383000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-7858a9ff-fdc3-4c99-9de8-ca40115370ec-0', usage_metadata={'input_tokens': 16, 'output_tokens': 53, 'total_tokens': 69})

In [3]:
model.invoke("Hello, how are you?")

AIMessage(content="I'm just a computer program, so I don't have feelings in the same way humans do. But thank you for asking! How can I assist you today?", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-12-17T19:03:23.760451212Z', 'done': True, 'done_reason': 'stop', 'total_duration': 278218954, 'load_duration': 20414852, 'prompt_eval_count': 16, 'prompt_eval_duration': 2000000, 'eval_count': 34, 'eval_duration': 255000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-5b551356-d94b-4e0f-9179-5c9747aa4617-0', usage_metadata={'input_tokens': 16, 'output_tokens': 34, 'total_tokens': 50})

In [7]:
messages = None
for index, chunk in enumerate(model.stream([HumanMessage("what color is the sky?")])):
    if messages is None:
        messages = chunk
    else:
        messages += chunk
    if index > 5:
        continue
    print(chunk)

print(messages)

content='The' additional_kwargs={} response_metadata={} id='run-74a34c4b-4c42-4a66-ba45-fe15108fcc5b'
content=' answer' additional_kwargs={} response_metadata={} id='run-74a34c4b-4c42-4a66-ba45-fe15108fcc5b'
content=' might' additional_kwargs={} response_metadata={} id='run-74a34c4b-4c42-4a66-ba45-fe15108fcc5b'
content=' seem' additional_kwargs={} response_metadata={} id='run-74a34c4b-4c42-4a66-ba45-fe15108fcc5b'
content=' simple' additional_kwargs={} response_metadata={} id='run-74a34c4b-4c42-4a66-ba45-fe15108fcc5b'
content=',' additional_kwargs={} response_metadata={} id='run-74a34c4b-4c42-4a66-ba45-fe15108fcc5b'
content="The answer might seem simple, but it actually depends on the time of day and atmospheric conditions.\n\n**During the daytime:**\nWhen the sun is out, the sky typically appears blue to our eyes. This is because the Earth's atmosphere scatters sunlight in all directions, with shorter (blue) wavelengths being scattered more than longer (red) wavelengths. This phenomeno